<center> <img src = https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/hh%20label.jpg alt="drawing" style="width:400px;">

# <font color='darkgreen'><center> **Проект: Анализ вакансий из HeadHunter**</font>
   

**Данный датасет представляет собой анализ вакансий с сайта по поиску работы hh.ru через базу данных project_sql.**

Также ознакомиться с проектом можно по следующей [**ссылке**](https://nbviewer.org/github/YaroslavaVob/DataScience_skillfactory/blob/eda832b181c9d55af5fad96ada1e0690734adbec/Project_2.%20%D0%90%D0%BD%D0%B0%D0%BB%D0%B8%D0%B7%20%D0%B2%D0%B0%D0%BA%D0%B0%D0%BD%D1%81%D0%B8%D0%B9%20%D1%81%D0%B0%D0%B9%D1%82%D0%B0%20HeadHunter/Project_2_Analysis%20of%20vacancies%20of%20site%20Headhunter.ipynb), в которой вы сможете наблюдать интерактивные графики некоторых признаков.

In [1]:
import pandas as pd
import psycopg2
import requests
from bs4 import BeautifulSoup
import plotly.express as px
from plotly.offline import plot, iplot, init_notebook_mode
init_notebook_mode(connected=True)
import warnings
warnings.filterwarnings('ignore')

In [2]:
connection = psycopg2.connect(
    dbname='unknown',
    user='unknown',
    host='unknown',
    password='unknown',
    port=5432
)

### <center><font color='green'>**1. Предварительный анализ данных**</font></center>

**1. Посмотрим на количество вакансий в нашей базе (*таблица vacancies*).**

In [3]:
# текст запроса
query_1_1 = f'''
    select count (*)    -- считаем количество всех вакансий
    from vacancies
    '''

In [4]:
# результат запроса
сount_vacancies = pd.read_sql_query(query_1_1, connection)
сount_vacancies

,count
0,49197


**2. Посчитаем количество работодателей (*таблица employers*).**

In [5]:
# текст запроса
query_1_2 = f'''
    select count (*)
    from employers
    '''

In [6]:
# результат запроса
count_employers = pd.read_sql_query(query_1_2, connection)
count_employers

,count
0,23501


**3. Проверим, сколько регионов и городов представлено в базе данных сайта (*таблица areas*).**

In [7]:
# текст запроса
query_1_3 = f'''
    select count (*)
    from areas
    '''

In [8]:
# результат запроса
count_cities = pd.read_sql_query(query_1_3, connection)
count_cities

,count
0,1362


**4. Также посчитаем количество сфер деятельности в базе данных (*таблица industries*).**

In [9]:
# текст запроса
query_1_4 = f'''
    select count (*)
    from industries
    '''

In [10]:
# результат запроса
count_industries = pd.read_sql_query(query_1_4, connection)
count_industries

,count
0,294


<font color ='orange'>**Выводы:**</font>
1. Предварительно взглянув на данные, представленные в таблицах базы данных HeadHunter, можем отметить, количество вакансий более, чем в два раза больше, чем работодателей, значит, у многих компаний не одна, а возможно несколько вакансий размещены на сайте.
2. Мы видим широкую географию городов и однозначно можно сказать, что сайт HeadHunter имеет популярность.
3. Сферы деятельности представлены почти 300 вариантами, это большое разнообразие.
___

### <center><font color='green'>**2. Детальный анализ вакансий**</font></center>

**1. Посчитаем, какое количество вакансий в каждом регионе. Выведем ТОП-10 городов по числу вакансий.**

In [11]:
# текст запроса
query_2_1 = f'''
    select distinct a.name as "Город",                       -- берем название города из таблицы areas
    count(v.id) as "Количество вакансий"                     -- подсчет количества по столбцу идентификаторов в таблице vacancies
    from areas as a join vacancies as v on a.id=v.area_id    -- объединяем столбцы таблицы areas и таблицы vacancies по id
    group by a.name                                          -- группируем по названию города
    order by 2 desc                                          -- сортируем по количеству в сторону убывания
'''

In [12]:
# результат запроса
count_vac_in_city = pd.read_sql_query(query_2_1, connection)
# выведем первые несколько строк таблицы
display(count_vac_in_city.head())

# отобразим ТОП-10 городов на графике
fig = px.bar(
    count_vac_in_city.iloc[:10],
    x = 'Город',
    y = 'Количество вакансий',
    title='ТОП-10 городов по количеству вакансий',
    height = 500,
    width = 800, 
    color_discrete_sequence = ['magenta']
)   
fig.update_layout(
    title_x = 0.5,
    title_font_size = 16,
    margin = dict(l=10, r=10, t=40, b=10),
    xaxis_title = 'Города'
)
iplot(fig)

,Город,Количество вакансий
0,Москва,5333
1,Санкт-Петербург,2851
2,Минск,2112
3,Новосибирск,2006
4,Алматы,1892


**2. База данных сайта содержит *вилку* в размере зарплаты: минимум и максимум.**\
**Посмотрим, у какого количества вакансий заполнен хотя бы один из двух признаков с зарплатой.**

In [13]:
# текст запроса
query_2_2 = f'''
    select count(*)
    from vacancies 
    where salary_from is not null or salary_to is not null   -- условие, где хотя один из признаков не является пустой ячейкой(null)
'''

In [14]:
# результат запроса
count_vac_with_salary = pd.read_sql_query(query_2_2, connection)
count_vac_with_salary

,count
0,24073


**3. Теперь найдем средние значения для нижней и верхней границы зарплатной *вилки*.**

In [15]:
# текст запроса
query_2_3 = f'''
    select round(avg(salary_from)) as "среднее значение минимальной зарплаты",
    round(avg(salary_to)) as "среднее значение предлагаемого максимума в зарплате"
    from vacancies
'''

In [16]:
# результат запроса
mean_low_high_salary = pd.read_sql_query(query_2_3, connection)
mean_low_high_salary

,среднее значение минимальной зарп,среднее значение предлагаемого ма
0,71065.0,110537.0


**4. Изучим, как тип рабочего графика (schedule) и тип трудоустройства (employment) влияет на количество вакансий.**


In [17]:
# текст запроса
query_2_4 = f'''
    select schedule,                      -- берем признак график работы и признак занятости из таблицы vacancies
    employment,
    count(*) as "Количество вакансий"     -- подсчитываем количество вакансий в группировке по признакам
    from vacancies
    group by schedule, employment
    order by 3 desc
'''

In [18]:
# результат запроса
schedule_employment_type = pd.read_sql_query(query_2_4, connection)
display(schedule_employment_type)

# отобразим на графике количество вакансий в зависимости от типа занятости и рабочего графика
fig = px.histogram(
    schedule_employment_type,
    x = 'Количество вакансий', 
    y = 'schedule',
    color = 'employment',
    orientation = 'h',
    height = 500,
    width = 1050,
    barmode = 'group',
    title = 'Распределение количества вакансий в зависимости от занятости и графика работы'
)
fig.update_layout(
    title_x = 0.5,
    title_font_size = 16,
    margin = dict(l=10, r=20, t=40, b=10),
    yaxis_title = 'Графики работы',
    legend_title = 'Виды занятости',
    xaxis_title = 'Количество вакансий',
    yaxis = dict(tickangle = 310)
)
iplot(fig)

,schedule,employment,Количество вакансий
0,Полный день,Полная занятость,35367
1,Удаленная работа,Полная занятость,7802
2,Гибкий график,Полная занятость,1593
3,Удаленная работа,Частичная занятость,1312
4,Сменный график,Полная занятость,940
5,Полный день,Стажировка,569
6,Вахтовый метод,Полная занятость,367
7,Полный день,Частичная занятость,347
8,Гибкий график,Частичная занятость,312
9,Полный день,Проектная работа,141


**5. Посмотрим, как растет число вакансий в зависимости от требуемого опыта работы (experience).** 

In [19]:
# текст запроса
query_2_5 = f'''
    select experience,
    count(*) "Количество вакансий"
    from vacancies
    group by experience
    order by 2
'''

In [20]:
# результат запроса
experience = pd.read_sql_query(query_2_5, connection)
display(experience)

# отобразим распределение на графике
fig = px.bar(
    experience,
    x = 'experience',
    y = 'Количество вакансий',
    title='Распределение количества вакансий в зависимости от опыта работы',
    height = 500,
    width = 800, 
    color_discrete_sequence = ['magenta']
)   
fig.update_layout(
    title_x = 0.5,
    title_font_size = 16,
    margin = dict(l=10, r=10, t=40, b=10),
    xaxis_title = 'Опыт работы'
)
iplot(fig)

,experience,Количество вакансий
0,Более 6 лет,1337
1,Нет опыта,7197
2,От 3 до 6 лет,14511
3,От 1 года до 3 лет,26152


<font color ='orange'>**Выводы:**</font>
1. По количеству вакансий среди городов абсолютно лидирует Москва, она в отрыве почти в два раза от Санкт-Петербурга и далее число вакансий только падает.
2. Только 24073 (почти половина) вакансий содержат информацию о заработной плате: стартовая и/или максимальная, нехороший факт для соискателя, ему придется или уточнять у работодателя будущую зарплату или он просто не будет рассматривать вакансию без указания зарплаты.
3. Средние показатели по зарплате: 71 ~ 110 тыс. рублей, лучше игнорировать, так как медиана или мода может очень сильно отличаться не только во всей выборке, но и между городами, для большей достоверности лучше всегда смотреть на медиану и моду по зарплате и в разных городах.
4. Самым востребованным типом занятости и типом трудойстройста являются полная занятость на полный рабочий день ~ 72% выборки.
5. Интересным фактом является опыт работы, мы видим, что большинство работадателей ищут сотрудников с опытом, но этот опыт минимален от 1 года до 3 лет, самым непопулярным оказался опыт от 6 лет, он очень сильно уступает категории "без опыта", которая примерно в 3,6 раз менее востребована, чем категория с опытом 1-3 года.
___

### <center><font color='green'>**3. Анализ работодателей**</font></center>

**1. Посмотрим на ТОП-10 работодателей по количеству вакансий.**

In [21]:
# текст запроса
query_3_1 = f'''
    select e.name "Работодатель",
    count(v.id) "Количество вакансий"
    from employers as e join vacancies as v on e.id = v.employer_id   -- объединяем таблицы empoyers(работадатели) и vacancies(вакансии)
    group by e.name                                                   -- группируем по работадателям
    order by 2 desc                                                   -- сортируем по убыванию
    limit 10                                                          -- отделяем 10 первых работодателей
'''

In [22]:
# результат запроса
employers = pd.read_sql_query(query_3_1, connection)
display(employers.iloc[:5])

# отобразим крупнейших работодателей на графике
fig = px.bar(
    employers,
    x = 'Работодатель',
    y = 'Количество вакансий',
    title='Топ-10 работодателей на рынке',
    height = 500,
    width = 800, 
    color_discrete_sequence = ['magenta']
)   
fig.update_layout(
    title_x = 0.5,
    title_font_size = 16,
    margin = dict(l=10, r=10, t=40, b=10),
    xaxis_title = 'Опыт работы'
)
iplot(fig)

,Работодатель,Количество вакансий
0,Яндекс,1933
1,Ростелеком,491
2,Тинькофф,444
3,СБЕР,428
4,Газпром нефть,331


**2. Выведем для каждого региона количество работодателей с учетом отсутствия вакансий для него в данном регионе.**

In [23]:
# текст запроса
query_3_2 = f'''
    select a.name "Регион",                                       -- берем названия регионов из таблицы areas
    count(distinct e.id) "Количество работодателей",              -- подсчитываем количество работадателей по таблице emploeyrs
    count(distinct v.id) "Количество вакансий"                    -- подсчитываем количество вакансий по таблице vacancies  
    From areas as a left join employers as e on a.id = e.area     -- объединение таблиц осуществляется по id
    left join vacancies as v on a.id=v.area_id                    
    where v.id is null                                            -- фильтруем только по пустым значениям вакансий 
    and e.id is not null                                          -- исключаем из таблицы работадателей пустые значения
    group by a.id                                                 -- группировка по id регионов, т.к. названия могут быть не уникальны
    order by 2 desc
'''

In [24]:
# результат запроса
city_count_vac_empoyers = pd.read_sql_query(query_3_2, connection)
city_count_vac_empoyers.head(20)


,Регион,Количество работодателей,Количество вакансий
0,Россия,410,0
1,Казахстан,207,0
2,Московская область,75,0
3,Краснодарский край,19,0
4,Беларусь,18,0
5,Ростовская область,18,0
6,Азербайджан,17,0
7,Нижегородская область,16,0
8,Республика Татарстан,16,0
9,Узбекистан,15,0


**3. Посчитаем для каждого работодателя количество регионов, в которых он публикует свои вакансии.**

In [25]:
# текст запроса
query_3_3 = f'''
    select e.name "Работодатель",
    count(distinct v.area_id) "Количество регионов"
    From employers as e join vacancies as v on e.id = v.employer_id
    group by e.id 
    order by 2 desc
'''

In [26]:
# результат запроса
employer_area = pd.read_sql_query(query_3_3, connection)
employer_area.head(20)

,Работодатель,Количество регионов
0,Яндекс,181
1,Ростелеком,152
2,Спецремонт,116
3,Поляков Денис Иванович,88
4,ООО ЕФИН,71
5,Совкомбанк,63
6,МТС,55
7,"ЭФКО, Управляющая компания",49
8,КРОН,48
9,Почта России,48


**4. Посмотрим, у какого количества работодателей не указана сфера деятельности.**

In [27]:
# текст запроса
query_3_4 = f'''
    select count(e.id) "Количество работодателей"
    From employers as e left join employers_industries as ei on e.id = ei.employer_id 
    left join industries as i on ei.industry_id = i.id           -- объединение трех таблиц по id через left join(объединение с сохранением пустых значений)
    where i.id is null                                           -- фильтрация по пустым ячейкам null в таблице industries(сферы деятельности)
'''

In [28]:
# результат запроса
employer_without_industries = pd.read_sql_query(query_3_4, connection)
employer_without_industries

,Количество работодателей
0,8419


**5. Посмотрим на компании, у которых указано четыре сферы деятельности.**

In [29]:
# текст запроса
query_3_5 = f'''
    select e.name "Работодатель"
    From employers as e join employers_industries as ei on e.id = ei.employer_id
    Group by e.id                                         -- используем для фильтрации агрегирующую функцию count(количество) группировочного признака id,
    having count(ei.industry_id) = 4                      -- так как e.name (название работадателя) может быть не уникальным
    order by 1           
'''

In [30]:
# результат запроса
employer_with_industries = pd.read_sql_query(query_3_5, connection)
employer_with_industries.head(15)

,Работодатель
0,101 Интернет
1,21vek.by
2,2ГИС
3,2К
4,4 пикселя +
5,5lb
6,5 Армия
7,741 Studios
8,AAEngineering Group
9,ADAPTER


**6. Проверим, у какого количества работодателей в качестве сферы деятельности указана "Разработка программного обеспечения".**


In [31]:
# текст запроса
query_3_6 = f'''
    select count(e.id) "Работодатель"
    From employers as e join employers_industries as ei on e.id = ei.employer_id
    join industries as i on ei.industry_id = i.id                   -- фильтрация сфер деятельности (i.name) с помощью функции like(ilike не чувствительна к регистру),
    where i.name ilike 'Разработка программного обеспечения'        -- которая находит искомый pattern в значениях признака
    '''                               

In [32]:
# результат запроса
employers_software_development = pd.read_sql_query(query_3_6, connection)
employers_software_development

,Работодатель
0,3553


**7. Для городов-миллионников России изучим количество вакансий от лидера работодателей - компании "Яндекс".**

Список городов-милионников получен [отсюда](https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8). 

In [33]:
# извлечем список городов-миллионников c сайта
url = 'https://ru.wikipedia.org/wiki/Города-миллионеры_России#Список_городов-миллионеровв'
response = requests.get(url, headers = {'User-Agent': 'Mozilla/5/0'})
soup = BeautifulSoup(response.text, 'html.parser')
# города-миллионники на веб-странице находятся в таблице по тегу <table>
cities = soup.find('table', class_='standard')
# создаем пустой список, где будем сохранять города
mln_cities = []
# в цикле извлекаем названия всех городов-миллионников из тега <a> по ключевому слову 'title', которые содержат названия городов
for city in cities.find_all('a'):
    mln_cities.append(city.get('title')) 
# преобразуем в кортеж для дайльнейшего использования в sql-запросе
mln_cities = tuple(mln_cities[4:])
print(mln_cities)   

('Москва', 'Санкт-Петербург', 'Новосибирск', 'Екатеринбург', 'Казань', 'Нижний Новгород', 'Красноярск', 'Челябинск', 'Самара', 'Уфа', 'Ростов-на-Дону', 'Краснодар', 'Омск', 'Воронеж', 'Пермь', 'Волгоград')


In [34]:
# текст запроса
query_3_7 = f'''
    (select a.name "Города",
    count(v.id) "Количество вакансий"
    From vacancies as v join areas as a on a.id = v.area_id       -- объединяем таблицы регионов(areas), вакансий(vacancies) 
    join employers as e on e.id = v.employer_id                   -- и работодателей (employers) по идентификаторам
    Where e.name = 'Яндекс' and a.name in {mln_cities}            -- проверяем нахождение города в списке городов-миллионников и фильтруем по работадателю "Яндекс"
    Group by a.id                                                 -- группируем по уникальным id городов
    Order by 2)
    Union all                                                     -- присоединяем таблицу уже с общим количеством по тем же параметрам
    (select 'Total',
    count(*)
    From vacancies as v join areas as a on a.id = v.area_id
    join employers as e on e.id = v.employer_id
    Where e.name = 'Яндекс' and a.name in {mln_cities}) 
'''

In [35]:
yandex_vacancies = pd.read_sql_query(query_3_7, connection)
yandex_vacancies

,Города,Количество вакансий
0,Total,485
1,Омск,21
2,Челябинск,22
3,Красноярск,23
4,Волгоград,24
5,Ростов-на-Дону,25
6,Казань,25
7,Пермь,25
8,Самара,26
9,Уфа,26


<font color ='orange'>**Выводы:**</font>
1. Абсолютным лидером по количеству вакансий является компания "Яндекс" и география ее публикаций самая широкая - 181 регион: как внутри России, так и за ее пределами (ближнее зарубежье).
2. Мы изучили работодателей и видим, что встречаются работодатели, которые ищут сотрудников вне своего региона.
3. Большой процент 35,8% работадателей не указывают в данных свою сферу деятельности, а вот у 15% компаний в сферу деятельности включена 'Разработка программного обеспечения', что говорит о росте на рынке IT-сферы.
___

### <center><font color='green'>**4. Предметный анализ**</font></center>

**1. Посмотрим, сколько вакансий имеет отношение к данным, то есть в названии вакансии присутствуют слова 'data' или 'данн'.**

In [36]:
# текст запроса
query_4_1 = f'''
    select count(*)
    from vacancies 
    where name ilike '%data%' or name ilike '%данн%'     -- фильтруем вакансии по pattern через функцию ilike(не чувствительную к регистру)
'''

In [37]:
# результат запроса
count_vac_data = pd.read_sql_query(query_4_1, connection)
count_vac_data

,count
0,1771


**2. Определим количество подходящих вакансий для начинающего дата-сайентиста(Junior)** по следующим критериям:
+ в названии вакансии для Data Scientist есть хотя бы одно из следующих сочетаний:
  * 'data scientist'
  * 'data science'
  * 'исследователь данных'
  * 'ML' (не включая вакансии по HTML)
  * 'machine learning'
  * 'машинн%обучен%'
+ Критерии для Junior будут следующими:
  * в названии есть слово 'junior' *или*
  * требуемый опыт — "Нет опыта" *или*
  * тип трудоустройства — "Стажировка".

In [38]:
# текст запроса
query_4_2 = f'''
    select count(*)
    from vacancies as v
    where (name ilike '%data scientist%'
    or name ilike '%data science%'
    or name ilike '%исследователь данных%'
    or (name like '%ML%' and name not like '%HTML%')
    or name ilike '%machine learning%'
    or name ilike '%машинн%обучен%')
    and(name ilike '%junior%' or lower(experience) = 'нет опыта' or lower(employment) = 'стажировка')
'''

In [39]:
# результат запроса
count_vacancies_ds = pd.read_sql_query(query_4_2, connection)
count_vacancies_ds

,count
0,51


**3. Посчитаем количество вакансий для Data Scientist, в которых в качестве ключевого навыка указан SQL или postgres.**

In [40]:
# текст запроса
query_4_3 = f'''
    select count(*)
    from vacancies
    where (name ilike '%data scientist%'
    or name ilike '%data science%'
    or name ilike '%исследователь данных%'
    or (name like '%ML%' and name not like '%HTML%')
    or name ilike '%machine learning%'
    or name ilike '%машинн%обучен%')
    and (key_skills ilike '%SQL%' or key_skills ilike '%postgres%')
'''

In [41]:
# результат запроса
count_vac_ds_skills = pd.read_sql_query(query_4_3, connection)
count_vac_ds_skills

,count
0,201


**4. Проверим, является ли востребованным знание ЯП Python в требованиях работодателей к Data Scientist.**

In [42]:
# текст запроса
query_4_4= f'''
    select count(*)
    from vacancies
    where (name ilike '%data scientist%'
    or name ilike '%data science%'
    or name ilike '%исследователь данных%'
    or (name like '%ML%' and name not like '%HTML%')
    or name ilike '%machine learning%'
    or name ilike '%машинн%обучен%')
    and (key_skills ilike '%python%')
'''

In [43]:
# результат запроса
count_vac_python_skills = pd.read_sql_query(query_4_4, connection)
count_vac_python_skills

,count
0,351


**5. Посчитаем, сколько ключевых навыков в среднем указывают в вакансиях для DS.**

In [44]:
"""
Для подсчета количества определенных символов в строке(тип text) из длины исходной строки вычтем длину строки, 
в которой все вхождения этого символа заменены на пустую строку ('') с помощью функции replace.
Если в навыках(key_skills) указано более одного, то заменим и символы табуляции (CHR(9)) на пустую строку ('').
Исключим из расчетов данные, в которых не указаны ключевые навыки(NULL).
Вычисление среднего значения сделаем с помощью агрегирующей функции avg, округлим значение до сотых с помощью функции round.
"""
# текст запроса
query_4_5= f'''
    select round(avg(length(key_skills) - length(replace(key_skills, CHR(9),''))+1), 2) as "Количество навыков для вакансии DS"
    from vacancies
    where (name ilike '%data scientist%'
    or name ilike '%data science%'
    or name ilike '%исследователь данных%'
    or (name like '%ML%' and name not like '%HTML%')
    or name ilike '%machine learning%'
    or name ilike '%машинн%обучен%')
    and key_skills is not null
'''

In [45]:
# результат запроса
count_skill_ds = pd.read_sql_query(query_4_5, connection)
count_skill_ds

,Количество навыков для вакансии DS
0,6.41


**6. Вычислим среднюю зарплату для Data Scientist для каждого типа требуемого опыта (experience).**

In [46]:
"""
Для расчета среднего по зарплате с двумя показателями: минимальная и максимальная, складываем оба признака и делим на два.
Для зарплат, где указан только один признак, второй заменяем на тот же показатель с помощью функции coalesce,
которая возвращает первый попавший аргумент, отличный от NULL.
И обязательно в условиях прописываем исключение зарплат, в которых оба признака = NULL
"""
# текст запроса
query_4_6= f'''
    select experience as "Требуемый опыт работы",
    round(avg(coalesce((salary_from + salary_to)/2, salary_from, salary_to))) as "Средняя зарплата"
    from vacancies
    where (name ilike '%data scientist%'
    or name ilike '%data science%'
    or name ilike '%исследователь данных%'
    or (name like '%ML%' and name not like '%HTML%')
    or name ilike '%machine learning%'
    or name ilike '%машинн%обучен%')
    and (salary_from is not null or salary_to is not null)
    Group by 1                                                  -- группируем полученные средние значения по опыту работы
'''

In [47]:
# результат запроса
experience_salary = pd.read_sql_query(query_4_6, connection)
display(experience_salary)

# отобразим на графике разницу в уровне зарплат в зависимости от опыта работы
fig = px.bar(
    experience_salary,
    x = "Требуемый опыт работы",
    y = "Средняя зарплата",
    title = 'Средний уровень заработной платы для специалиста Data Science',
    height = 500,
    width = 700,
    color_discrete_sequence = ['magenta']
)
fig.update_layout(
    title_x = 0.5,
    title_font_size = 16,
    margin = dict(l=10, r=10, t=40, b=10)
)
iplot(fig)

,Требуемый опыт работы,Средняя зарплата
0,Нет опыта,74643.0
1,От 1 года до 3 лет,139675.0
2,От 3 до 6 лет,243115.0


**7. Выделим ТОП-20 компаний, которые ищут специалистов в сфере Data Science.**

In [48]:
# текст запроса
query_4_7 = f'''
    (select e.name as "Работодатель",
    count(v.id) "Количество вакансий"
    from employers as e                                      -- объединяем таблицу employers(работадатели) и таблицу vacancies(вакансий)
    Join vacancies as v on e.id = v.employer_id              -- по идентификаторам
    where (v.name ilike '%data scientist%'                   -- фильтруем по тем же признакам, относящимся к Data Science
    or v.name ilike '%data science%'
    or v.name ilike '%исследователь данных%'
    or (v.name like '%ML%' and v.name not like '%HTML%')
    or v.name ilike '%machine learning%'
    or v.name ilike '%машинн%обучен%')
    Group by 1                                               -- группируем по компаниям
    Order by 2 desc
    Limit 20)
    Union all                                                -- присоединяем таблицу в общим количеством вакансий
    (Select 'Total',                                         
    count(v.id)                                              -- посчитаем общее количество вакансий(v.id) в сфере Data Science
    from employers as e                                      
    Join vacancies as v on e.id = v.employer_id
    where (v.name ilike '%data scientist%'
    or v.name ilike '%data science%'
    or v.name ilike '%исследователь данных%'
    or (v.name like '%ML%' and v.name not like '%HTML%')
    or v.name ilike '%machine learning%'
    or v.name ilike '%машинн%обучен%'))
    '''

In [49]:
# результат запроса
employer_for_ds = pd.read_sql_query(query_4_7, connection)
display(employer_for_ds)
connection.close()

,Работодатель,Количество вакансий
0,СБЕР,37
1,Bell Integrator,25
2,Банк ВТБ (ПАО),18
3,VK,15
4,Positive Technologies,11
5,Яндекс,9
6,EvenBet Gaming,9
7,МегаФон,8
8,Andersen,7
9,МТС,6


<font color ='orange'>**Выводы:**</font>
1. Востребованность специалистов в сфере data постепенно растет на рынке, 1771 вакансия относится к ним, 27% (480 вакансий) из них в направлении Data Science.
2. Одним из самых популярных навыков, который ждут от Data Scientist является знание Python.
3. Отметим хорошую стартовую зарплату для Data Scientist даже без опыта работы в среднем ~ 75 тыс. руб. и высокую для опытного специалиста ~ 243 тыс. руб.
4. Крупнейшие и богатейшие компании страны ищут специалистов Data Science, это нам говорит о том, что для дальнейшего роста и успеха бизнеса данные специалисты становятся ключевыми сотрудниками.
___

## <font color='darkgreen'><center>**Заключение по проекту**</center></font>
1. Сайт HeadHunter является очень популярной площадкой по поиску работы и предоставляет достаточно много данных для исследований в сфере рынка труда и заработной платы, роста и развития сфер деятельности.
2. Мы выявили крупнейших работодателей на рынке вакансий и убедились, что крупнейшие успешные компании ищут специалистов Data Science. Значит, рост и популярность профессии Data Scientist является для них показателем дальнейшего развития и необходимым фактором. 
3. Даже стартовая зарплата специалиста Data Science выше медианной зарплаты по стране, а специалист с опытом получает одну из лучших зарплат по стране (с исследованием, где можно ознакомиться с данными по зарплатам [**здесь**](https://github.com/YaroslavaVob/DataScience_skillfactory/blob/main/Project_1.%20%D0%90%D0%BD%D0%B0%D0%BB%D0%B8%D0%B7%20%D1%80%D0%B5%D0%B7%D1%8E%D0%BC%D0%B5%20%D0%B8%D0%B7%20hh/Project-1._Data%20Analysis%20of%20resumes%20(hh.ru).ipynb))).
3. Среди профессиональных навыков для Data Scientist лидируют Python, SQL, postgres.
5. Одно из направлений IT-сфер 'разработка программного обеспецения' входит в сферу деятельности 15% компаний, да, указание данного направления не является обоснованием утверждать, что компания этим точно занимается, но явно прослеживается, что все большие и успешные компании сегодня ищут сотрудников IT сферы и растет вотребованность в Data Scientist-специалистах.

<font color ='orange'>**Примечание:**</font>\
Для исследования была взята досточно большая выборка вакансий на рынке и проведен анализ с уклоном на изучение роста популярности профессии Data Scientist.\
Исследование можно продолжить как в этом направлении, так и изучить более детально остальные данные: например, работадатели с какими сферами деятельности ищут специалистов на сайте hh.ru, где зарегистрированы большинство работадателей (география уплаты налогов), посмотреть критерии заработной платы в других профессиях и т.д.\
После моего плодотворного труда, принимаю любые дополнительные исследования и комментарии по проекту.
<center><img src=https://masterpiecer-images.s3.yandex.net/8c23d6ed716911eeadf9beb332dff282:upscaled width=30% height=30%>